# Separar os dados de ponto médio do quadril com dados de MoCap

In [2]:
from utils import *
%load_ext autoreload
%autoreload 2
%matplotlib inline

# File location

v3d_filename = r'C:\Users\gusta\OneDrive\Documentos\Users\Gustavo Monteiro\Documents\GitHub\MoVi-Toolbox\MoCap\F_v3d_Subject_15.mat'

# Reading MoVi files as a Python dictionary
sample = mat2dict(v3d_filename)

# First move sample
move = sample['move']

move =dict2ntuple(move)

print(move.markerLocation)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[[[ 336.98776245   17.4847908  1559.65722656]
  [ 307.49227905   20.96870995 1235.98718262]
  [ 466.79177856   24.70891571 1497.18261719]
  ...
  [ 356.89865112 -166.20855713 1462.76721191]
  [ 487.96240234 -695.30969238 1474.58483887]
  [ 535.75872803  -89.18184662   11.15608311]]

 [[ 336.91867065   17.47442055 1559.62512207]
  [ 307.44500732   20.97799873 1236.01245117]
  [ 466.72747803   24.72114944 1497.20153809]
  ...
  [ 356.89349365 -166.20951843 1462.75097656]
  [ 488.02035522 -695.31994629 1474.49694824]
  [ 535.76281738  -89.28752136   11.33196163]]

 [[ 336.85165405   17.46774292 1559.61340332]
  [ 307.38452148   20.97657204 1236.02563477]
  [ 466.66177368   24.71905136 1497.2244873 ]
  ...
  [ 356.87591553 -166.23568726 1462.76635742]
  [ 488.07287598 -695.32769775 1474.41381836]
  [ 535.77148438  -89.28872681   11.34207153]]

 ...

 [[ 249.93983459   39.77682495 1558.72497559]
  [ 218.

--> Salvar os dados no arquivo marker data

In [3]:
np.save('marker data', move.markerLocation)

--> Carregar os dados

In [4]:
array = np.load('marker data.npy')

--> Calcular os pontos médios e armazenar os dados em um dataframe

In [6]:
import cv2
import numpy as np
from numpy import asarray
from numpy import save
import pandas as pd
import matplotlib.pyplot as plt
import seaborn; seaborn.set()

left_hip_joint = array[:,73, :]
right_hip_joint = array[:,81,:]

pmx = (left_hip_joint[:,0] + right_hip_joint[:,0])*0.5
pmy = (left_hip_joint[:,1] + right_hip_joint[:,1])*0.5
pmz = (left_hip_joint[:,2] + right_hip_joint[:,2])*0.5

pm = np.array([pmx, pmy, pmz])

df = pd.DataFrame(pm,
                 index=['X', 'Y', 'Z'])

df_t = df.T

df_t

,X,Y,Z
0,376.391708,22.871300,975.676544
1,376.373230,22.885738,975.683594
2,376.359344,22.907471,975.685211
3,376.381714,22.896675,975.652802
4,376.277695,22.877411,975.643372
...,...,...,...
17841,290.633636,52.762039,978.335938
17842,290.668442,52.771891,978.338013
17843,290.726730,52.753082,978.333282
17844,290.770691,52.762125,978.340973


--> Salvar no arquivo Origin data. Foi dado esse nome pois o ponto médio dos quadris no Blaze Pose representa a origem do sistema.

In [9]:
np.save('Origin data', df_t)

--> Rode os códigos abaixo para chegar ao dataframe com os dados dos pontos médios para todos os frames.

In [10]:
import os
import collections
import re
import cv2

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import scipy.io as sio
import scipy.signal as signal
from pathlib import Path
from sources_cv2 import defineStartEnd, count_frames


In [11]:
# Caminho onde estão os dados .mat
caminho = r'C:\\Users\\gusta\\OneDrive\\Documentos\\Users\\Gustavo Monteiro\\Documents\\GitHub\\MoVi-Toolbox\\MoCap\\'

In [12]:
# Número do arquivo do voluntário, alterar apenas o número

''' Alterar o número do sujeito analisado'''

voluntary = 15

In [13]:
voluntary = str(voluntary)
v3d_filename = caminho + 'F_v3d_Subject_' + voluntary + '.mat'

In [14]:
array = np.load('Origin data.npy')

In [15]:
Data = array
name_description = move.motions_list
data_complete = pd.DataFrame(Data,
                           columns = ['X', 'Y', 'Z'])

In [16]:
video_path = r'C:\\Users\\gusta\\OneDrive\\Documentos\\Users\\Gustavo Monteiro\\Documents\\GitHub\\MoVi-Toolbox\\MoCap\\'

video_F = 'F'

In [17]:
# Frequencia dos dados

video_name = video_path + video_F + '_PG1_Subject_' + voluntary + '_L.avi'

freq_MoCap = 120
n_frames, fps = count_frames(video_name)

In [17]:
min_init_video = 1  
seg_init_video = 55

aprox_init_video = int(((min_init_video * 60) + seg_init_video) * fps)

In [18]:
start_end = defineStartEnd(video_name, n_frames, aprox_init_video)

[3463, 0]
[3463, 3503]


In [19]:
frame_start = start_end[0]
frame_end = start_end[1]

temp_start = frame_start / fps
MOCAP_start = int(temp_start * freq_MoCap)

temp_end = frame_end / fps
MOCAP_end = int(temp_end * freq_MoCap)

data_pm = data_complete.iloc[MOCAP_start:MOCAP_end + 1,:]

In [20]:
data_pm

,X,Y,Z
13852,1015.728394,1100.592163,955.580170
13853,1018.811432,1093.375122,954.166687
13854,1021.897308,1086.352417,952.833069
13855,1024.911072,1079.041016,951.384247
13856,1028.022278,1071.547974,950.097504
...,...,...,...
14008,979.134521,-325.063782,939.651917
14009,978.134888,-334.936111,939.325958
14010,976.841980,-344.724213,938.876709
14011,975.315613,-354.700302,939.001770
